In [ ]:
# Reset and Restart 

import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
reload(DisplayWindows)

CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()

ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

%matplotlib qt

# Automation Objects

In [ ]:
# Set up the Automation Method & Template 
TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300CellChamber.txt"
METHOD_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\Brae\SingleCell.txt"

from L4 import AutomatedControl
from L4 import Trajectory
reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)
auto_program.add_method(METHOD_PATH)
auto_program.repetitions=6

In [ ]:
# Start the Method
if not auto_program.continue_event.is_set():
    auto_program.continue_event.set()
else:
    auto_program.start_run()

In [ ]:
# Stop the Method
auto_program.stop_run()

In [ ]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()

# Set the Capillary Adjustments

In [ ]:
# Move the objective to the height and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(+.05)

In [ ]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height

def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

# Find the Brightest Cell

In [ ]:
# PARAMETERS
target_x = 695
target_y = 485
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 250
_old_exp=100
_old_bin = 1
_old_channel = 0
bins = 8

In [ ]:
# SNAP a FLUOR IMAGE
# Set to new values
ce_system.inlet_rgb.turn_off_channel('R')
ce_system.camera.stop()
ce_system.filter_wheel.set_channel(fluor_channel)
ce_system.excitation_shutter.set_auto_on()
#_old_exp = ce_system.camera.exposure
ce_system.camera.set_exposure(exp)
ce_system.camera.set_binning(bins)
#_old_bin = ce_system.camera.bin_size
time.sleep(wait)

img = ce_system.camera.snap()

# Return to old values
ce_system.camera.set_exposure(_old_exp)
ce_system.camera.set_binning(_old_bin)
ce_system.excitation_shutter.set_auto_off()
ce_system.filter_wheel.set_channel(_old_channel)
ce_system.inlet_rgb.turn_on_channel('R')

In [ ]:
close_size=5
open_size=5
watershed_footprint = (5,5)
# Make sure types are the same
input_image = img_as_float(img)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(filtered_image)
binary_otsu = filtered_image > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

if df.shape[0] > 0:
    y=df['centroid-0'][0]*bins
    x=df['centroid-1'][0]*bins
    
    movex = (x-target_x)*0.0001063
    movey = (y-target_y)*0.0001063
    ce_system.xy_stage.set_rel_x(movex)
    ce_system.xy_stage.set_rel_y(-movey)


# Single Cell Injection/Load 

In [ ]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    print(obj_height)
    cap_z = get_cap_height(obj_height, cap_difference)
    z = ce_system.inlet_z.read_z()
    rel_z = cap_z-z
    rel_z += 0.02
    ce_system.inlet_z.set_rel_z(rel_z)
else:
    print("Bring cells into focus.")

In [ ]:
# Raise the objective for optimal plasma formation
ce_system.objective.set_rel_z(+0.015)

In [ ]:
# Lower the capillary (ADJUSTMENT)
ce_system.inlet_z.set_rel_z(-0.1)

In [ ]:
# General Injection Parameters
injection_time = 5 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 10 # in kilovolts
delay = 1 # inseconds

In [ ]:
# Start the Single Cell Injection
import threading

def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    while time.time()-start_time < injection_time:
        time.sleep(0.01)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    

threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

In [ ]:
ce_system.inlet_z.set_rel_z(5)

# PUT YOUR CRAP FUNCTIONS HERE

In [ ]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [ ]:
# Stop Rinse 
ce_system.outlet_pressure.release()